# Signal Editor

#### TODO
Adicionar uma feature de pop intervalo <br>

## Importando módulos

In [2]:
import os
import ipywidgets as widgets
from intervals import Intervals
from signalPlotter import SignalPlotter

## Variáveis globais para o diretório dos dados e os canais

In [3]:
files_path = '../../../dataset_eeg_cafe2022/RenamedFiles'
channels = ['Fp1', 'Fp2', 'C3', 'C4', 'P7', 'P8', 'O1', 'O2']

## Recuperando arquivos de sinal

In [4]:
files = [file for file in os.listdir(files_path) if '.csv' in file]

if len(files) == 0:
    print('Sem dados no diretório especificado.')
    exit()

## Crinado flags para o estado dos widgets de arquivo e canal

In [5]:
file_widget_changed = False
channel_widget_changed = False

## Criando widget de output

In [6]:
output = widgets.Output()

## Criando instâncias das classes SignalPlotter e Intervals

In [7]:
intervalsObject = Intervals(files_path)
signalPlotterObject = SignalPlotter(files_path,output)

## Criando os widgets

#### Widgets para configurar o sinal

In [8]:
file_widget = widgets.Combobox(options = files, value = files[0], description = 'Arquivo')
channel_widget = widgets.Dropdown(options = channels, value = channels[0], description = 'Canal')
refreshButton_widget = widgets.Button(description = 'Refresh', layout = widgets.Layout(width = '70px'))

signalControler_widget = widgets.HBox([file_widget, channel_widget, refreshButton_widget])

#### Widgets para lidar com os intervalos de corte

In [9]:
label_widget = widgets.Label(value = 'Intervalo de corte')
bottonLimit_widget = widgets.FloatText(layout = widgets.Layout(width='100px'))
topLimit_widget = widgets.FloatText(layout = widgets.Layout(width='100px'))
addIntervalButton_widget = widgets.Button(description = 'Add', layout = widgets.Layout(width = '50px'))
saveIntervalButton_widget = widgets.Button(description = 'Save', layout = widgets.Layout(width = '50px'))
#Criar um pop interval

choppingIntervals_widget = widgets.HBox([label_widget, bottonLimit_widget, topLimit_widget, addIntervalButton_widget, saveIntervalButton_widget])

## Definindo eventos

#### Handlers para monitorar os widgets de arquivo e canal

In [10]:
def file_widget_change_handler(change):
    global file_widget_changed
    if change['type'] == 'change' and change['name'] == 'value':
        file_widget_changed = True

In [11]:
def channel_widget_change_handler(change):
    global channel_widget_changed
    if change['type'] == 'change' and change['name'] == 'value':
        channel_widget_changed = True 

#### Eventos de click

In [12]:
def refresh_button_clicked(btn):
    global file_widget_changed
    global channel_widget_changed
    # é possível validar dados antes de alterar os objetos
    if file_widget_changed:
        intervalsObject.save_cutting_intervals()
        intervalsObject.load_cutting_intervals(file_widget.value)
        signalPlotterObject.load_signal(file_widget.value, channel_widget.value)
        signalPlotterObject.plot_signal(intervalsObject.get_cutting_intervals(channel_widget.value))
        file_widget_changed = False
    elif channel_widget_changed:
        signalPlotterObject.change_current_fig(channel_widget.value)
        signalPlotterObject.plot_signal(intervalsObject.get_cutting_intervals(channel_widget.value))
        channel_widget_changed = False

In [13]:
def add_intervals_clicked(btn):
    start = bottonLimit_widget.value
    end = topLimit_widget.value
    intervalsObject.add_interval(channel_widget.value, start, end)
    signalPlotterObject.plot_signal(intervalsObject.get_cutting_intervals(channel_widget.value))

In [14]:
def save_intervals_clicked(btn):
    intervalsObject.save_cutting_intervals()

## Associando widgets aos seus eventos

In [15]:
file_widget.observe(file_widget_change_handler)
channel_widget.observe(channel_widget_change_handler)

refreshButton_widget.on_click(refresh_button_clicked)
addIntervalButton_widget.on_click(add_intervals_clicked)
saveIntervalButton_widget.on_click(save_intervals_clicked)

## Main

In [16]:
intervalsObject.load_cutting_intervals(file_widget.value)
signalPlotterObject.load_signal(file_widget.value, channel_widget.value)
signalPlotterObject.plot_signal(intervalsObject.get_cutting_intervals(channel_widget.value))

display(signalControler_widget, output, choppingIntervals_widget)

Output()